This Notebook covers the web scraping and the exploratory data analysis. The sample of the output dataset named "Full_data" has been uploaded.

In [1]:
# import the necessnary libs
import pandas as pd
import numpy as np
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler
import urllib
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

In [2]:
import requests
URL = 'https://racing.hkjc.com/racing/information/English/Racing/ResultsAll.aspx?RaceDate=2010/09/05'
page = requests.get('https://racing.hkjc.com/racing/information/chinese/Racing/ResultsAll.aspx?RaceDate=2010/09/05')

In [3]:
# Solve the math game in the web page
from math import cos, pi, floor

import requests

def parse_challenge(page):
    """
    Parse a challenge given by mmi and mavat's web servers, forcing us to solve
    some math stuff and send the result as a header to actually get the page.
    This logic is pretty much copied from https://github.com/R3dy/jigsaw-rails/blob/master/lib/breakbot.rb
    """
    top = page.split('<script>')[1].split('\n')
    challenge = top[1].split(';')[0].split('=')[1]
    challenge_id = top[2].split(';')[0].split('=')[1]
    return {'challenge': challenge, 'challenge_id': challenge_id, 'challenge_result': get_challenge_answer(challenge)}


def get_challenge_answer(challenge):
    """
    Solve the math part of the challenge and get the result
    """
    arr = list(challenge)
    last_digit = int(arr[-1])
    arr.sort()
    min_digit = int(arr[0])
    subvar1 = (2 * int(arr[2])) + int(arr[1])
    subvar2 = str(2 * int(arr[2])) + arr[1]
    power = ((int(arr[0]) * 1) + 2) ** int(arr[1])
    x = (int(challenge) * 3 + subvar1)
    y = cos(pi * subvar1)
    answer = x * y
    answer -= power
    answer += (min_digit - last_digit)
    answer = str(int(floor(answer))) + subvar2
    return answer


def getPage(URL):
  if __name__ == '__main__':
    s = requests.Session()
    r = s.get(URL)

    if 'X-AA-Challenge' in r.text:
        challenge = parse_challenge(r.text)
        r = s.get(URL, headers={
            'X-AA-Challenge': challenge['challenge'],
            'X-AA-Challenge-ID': challenge['challenge_id'],
            'X-AA-Challenge-Result': challenge['challenge_result']
        })

        yum = r.cookies
        r = s.get(URL, cookies=yum)

    return r

In [4]:
import requests, re
from bs4 import BeautifulSoup
from urllib.parse import urljoin

table_source = "https://racing.hkjc.com/racing/information/English/Horse/HorseFormerName.aspx"
start_url = "https://racing.hkjc.com"

page = getPage(table_source)

In [5]:
# Extract the Link regarding the information of each horse
horse_info_links = []
soup = BeautifulSoup(page.text, 'html.parser')
table = BeautifulSoup(str(soup.find_all(class_='bigborder')))
links = table.find_all('a', href=re.compile("HorseId="))
for item in links:
    horse_info_links.append(urljoin(start_url, item['href']))
print(horse_info_links)
URL = horse_info_links[0]
name=URL.split("=")[1].split("_", 1)[1]
print(name)

['https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H037', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H342', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2020_E144', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H331', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2021_G196', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2023_J181', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H046', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H076', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2023_J328', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H082', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def horse_df(URL):
    # Extracting horse name from URL
    name = URL.split("=")[1].split("_", 1)[1]
  
    # Simulating your getPage function to fetch page content
    page = requests.get(URL)
    soup = BeautifulSoup(page.text, 'html.parser')
  
    # Extracting information from the HTML using BeautifulSoup
    data = {}
    tables = soup.find_all('table', class_='horseProfile')
  
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            if len(cells) == 3:  # key-value format
                key = cells[0].text.strip()
                value = cells[2].text.strip()
                data[key] = value
  
    # Special handling for "Same Sire" due to different HTML structure
    same_sire_select = soup.find('select', id='SameSire')
    if same_sire_select:
        options = [option.text.strip() for option in same_sire_select.find_all('option')]
        data['Same Sire'] = ' '.join(options)
  
    # Creating a single-row DataFrame
    df = pd.DataFrame([data])
  
    # Adding horse name column
    df['name'] = name
  
    # Splitting "Country of Origin / Age" into separate columns if present
    if 'Country of Origin / Age' in df.columns:
        df[['Country of Origin', 'Age']] = df['Country of Origin / Age'].str.split(' / ', expand=True)
        df.drop('Country of Origin / Age', inplace=True, axis=1)
  
    # Splitting "Colour / Sex" into separate columns if present
    if 'Colour / Sex' in df.columns:
        df[['Colour', 'Sex']] = df['Colour / Sex'].str.split(' / ', expand=True)
        df.drop('Colour / Sex', inplace=True, axis=1)

    # Inserting an unnamed column at the beginning
    #df.insert(0, '_unnamed', np.nan)
  
    return df


In [14]:
display(horse_df(URL))

,_unnamed,Import Type,Season Stakes*,Total Stakes*,No. of 1-2-3-Starts*,No. of starts in past 10race meetings,Current Stable Location(Arrival Date),Import Date,Trainer,Owner,...,Start ofSeason Rating,Sire,Dam,Dam's Sire,Same Sire,name,Country of Origin,Age,Colour,Sex
0,NaN,PPG,"$2,976,000","$4,619,550",4-1-4-24,1,Hong Kong(06/04/2024),05/07/2022,J Size,Chan Yee Lap,...,61,Per Incanto,Island Time,Al Akbar,J409 J421 AEROINVINCIBLE BABY CRYSTAL BRAVO WA...,2022_H037,NZ,5,Bay,Gelding


In [8]:
print(horse_info_links)

['https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H037', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H342', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2020_E144', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H331', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2021_G196', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2023_J181', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H046', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H076', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2023_J328', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H082', 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK

In [9]:
# read tables of all horses using the hourses_id extracted eariler V2
import concurrent.futures
import pandas as pd
import warnings
from time import time

warnings.filterwarnings("ignore")

# Assuming horse_df is a function that takes a link and returns a DataFrame

def process_link(link):
    try:
        df = horse_df(link)
        return df
    except Exception as e:
        print(f"Error processing link {link}: {e}")
        return None

def main(horse_info_links):
    start_time = time()
    df_list = []

    # Using ThreadPoolExecutor to parallelize I/O bound tasks
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(process_link, link) for link in horse_info_links]
        for i, future in enumerate(concurrent.futures.as_completed(futures)):
            result = future.result()
            if result is not None:
                df_list.append(result)
            print(f"Progress: {i+1}/{len(horse_info_links)}")
            print(f"Time used: {time() - start_time}")
            print("=" * 20)

    result_df = pd.concat(df_list, ignore_index=True)
    return result_df

# Replace horse_info_links with your list of links
result_df = main(horse_info_links)

result = pd.concat(df_list)
result = result.reset_index(drop=True)

Progress: 1/1229
Time used: 0.6871440410614014
Progress: 2/1229
Time used: 0.9852197170257568
Progress: 3/1229
Time used: 0.9852418899536133
Progress: 4/1229
Time used: 1.0269629955291748
Progress: 5/1229
Time used: 1.0792558193206787
Progress: 6/1229
Time used: 1.079265832901001
Progress: 7/1229
Time used: 1.0834870338439941
Error processing link https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2021_G196: Columns must be same length as key
Progress: 8/1229
Time used: 1.0854418277740479
Progress: 9/1229
Time used: 1.0875787734985352
Progress: 10/1229
Time used: 1.087587833404541
Progress: 11/1229
Time used: 1.4611358642578125
Progress: 12/1229
Time used: 2.0261456966400146
Progress: 13/1229
Time used: 2.0512959957122803
Progress: 14/1229
Time used: 2.0513060092926025
Error processing link https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2021_G269: Columns must be same length as key
Progress: 15/1229
Time used: 2.16693091392517

In [12]:
result_df.to_csv("horse_family.csv")
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Import Type                            1173 non-null   object
 1   Season Stakes*                         1173 non-null   object
 2   Total Stakes*                          1173 non-null   object
 3   No. of 1-2-3-Starts*                   1173 non-null   object
 4   No. of starts in past 10race meetings  1173 non-null   object
 5   Current Stable Location(Arrival Date)  1173 non-null   object
 6   Import Date                            1173 non-null   object
 7   Trainer                                1173 non-null   object
 8   Owner                                  1173 non-null   object
 9   Current Rating                         1173 non-null   object
 10  Start ofSeason Rating                  1173 non-null   object
 11  Sire             